### 여가시설 및 맛집 추천 시스템 (KNN & Decision Tree 기반) - 사용자 위치 기반

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# 1. 여가시설 및 맛집 데이터 불러오기
facilities = pd.read_csv('./data/인천광역시 시설 현황.csv', encoding='euc-kr')
restaurants = pd.read_csv('./data/인천광역시 식당 현황.csv', encoding='CP949')

In [3]:
facilities.head(1)

,시설명,공원유형,시설분류,도로명 주소,행정구역,lat,lon
0,수봉공원,근린공원,배드민턴장,인천광역시 미추홀구 주안동 660-241,미추홀구,37.460045,126.664129


In [4]:
restaurants.head(1)

,식당명,행정구역,도로명 주소,lat,lon,식당 설명,영업 시간,휴일 정보
0,편가네된장,강화군,인천광역시 강화군 화도면 가능포로89번길 11,37.640677,126.419384,강화 맛집. 편가네 된장. 4대째 내려오는 전통방식 그대로국내산 재료를 엄선해 화학...,"평일 10:00-20:00 / 주말,공휴일 09:00-20:00",매주 수요일(공휴일 제외) / 명절 당일 휴무


In [3]:
# 2. 시설 데이터 전처리
facilities = facilities[['시설명', '시설분류', '도로명 주소', 'lat', 'lon']].dropna()

# 3. 식당 데이터 전처리
restaurants = restaurants[['식당명', '행정구역', '도로명 주소', 'lat', 'lon']].dropna()
restaurants['시설분류'] = '식당'
restaurants.rename(columns={
    '식당명': '시설명'
}, inplace=True)

In [4]:
# 4. 데이터 통합
combined = pd.concat([facilities, restaurants], ignore_index=True)

In [5]:
# 5. 범주형 인코딩
le_type = LabelEncoder()
combined['시설분류_encoded'] = le_type.fit_transform(combined['시설분류'])

In [6]:
# 6. 피처 및 타겟 정의
X = combined[['시설분류_encoded', 'lat', 'lon']]
y = combined['시설명']

In [7]:
# 7. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
facilities['시설분류'].unique()

array(['배드민턴장', '게이트볼장', '농구장', '풋살장', '야외운동기구', '인라인스케이트장', '족구장',
       '다목적구장', '축구장', '국궁장', '다목적운동장', '테니스장', '운동장', '소운동장', '야구장',
       'X-게임장'], dtype=object)

In [13]:
# 8. 사용자 선호 정보 (식당 + 여가시설)
user_preferences = [
    {'시설분류': '식당', 'lat': 37.5, 'lon': 126.7},
    {'시설분류': '축구장', 'lat': 37.5, 'lon': 126.7}
]

In [14]:
# 9. 추천 수행
all_knn_results = []
all_tree_results = []

for pref in user_preferences:
    if pref['시설분류'] not in le_type.classes_:
        print(f"❌ '{pref['시설분류']}'은(는) 시설분류에 존재하지 않아요.")
        continue

    user_type = le_type.transform([pref['시설분류']])[0]

    user_input_df = pd.DataFrame([[
        user_type,
        pref['lat'],
        pref['lon']
    ]], columns=['시설분류_encoded', 'lat', 'lon'])

    user_input_scaled = scaler.transform(user_input_df)

    # KNN 추천
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_scaled, y)
    knn_indices = knn.kneighbors(user_input_scaled, return_distance=False)[0]
    knn_results = combined.iloc[knn_indices][['시설명', '시설분류', '도로명 주소']]
    all_knn_results.append((pref['시설분류'], knn_results))

    # 결정 트리 추천
    tree = DecisionTreeClassifier(max_depth=5)
    tree.fit(X, y)
    tree_prediction = tree.predict(user_input_df)
    all_tree_results.append((pref['시설분류'], tree_prediction[0]))

C:\Users\406\miniconda3\envs\yh\lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
C:\Users\406\miniconda3\envs\yh\lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
C:\Users\406\miniconda3\envs\yh\lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
C:\Users\406\miniconda3\envs\yh\lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater 

In [15]:
# 10. 결과 출력
for category, results in all_knn_results:
    print(f"\n KNN 추천 결과 for '{category}':")
    print(results)

for category, prediction in all_tree_results:
    print(f"\n 결정 트리 추천 for '{category}': {prediction}")


 KNN 추천 결과 for '식당':
                  시설명 시설분류             도로명 주소
798               황가복   식당     인천 부평구 마장로 245
620        남대감꽃게아구전문점   식당     인천 부평구 마장로 243
809             명품삼계탕   식당     인천 부평구 마장로 253
759     우리콩 우리쌀 현미순두부   식당     인천 부평구 마장로 265
1006  양평신내서울해장국 부평직영점   식당  인천광역시 부평구 마장로 191

 KNN 추천 결과 for '축구장':
        시설명 시설분류                도로명 주소
102  신트리공원   축구장     인천광역시 부평구 신트리로 21
64     효성공원  축구장   인천광역시 계양구 새벌로111번길7
66     작전공원  축구장  인천광역시 계양구 아나지로307번길1
124    구월공원  축구장   인천광역시 남동구 구월2동 1246
50     계산공원  축구장     인천광역시 계양구 주부토로570

 결정 트리 추천 for '식당': 금강산추어탕

 결정 트리 추천 for '축구장': 구월공원
